In [ ]:
import os
import torch
import numpy as np
import plotly
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import plotly.express as px
# px.defaults.template = "ggplot2"

from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.optim import optimize_acqf
import math

In [ ]:
def rosenbrock_2d(x_ax, y_ax, x_ax_len=10, y_ax_len=10):
    # max (x,y) = (1,1)
    full_result = []
    for x in x_ax.reshape((x_ax_len, -1)):
        row_result = []
        for y in x_ax.reshape((y_ax_len, -1)):
            row_result.append(
                -((1-x[0])**2 + 0.03*(y[0] - x[0]**2)**2)+5
            )
        full_result.append(row_result)
    return torch.tensor(full_result)

def bellcurve_2d(x_ax, y_ax, x_ax_len=10, y_ax_len=10):
    # max (x,y) = (0,0)
    full_result = []
    for x in x_ax.reshape((x_ax_len, -1)):
        row_result = []
        for y in x_ax.reshape((y_ax_len, -1)):
            row_result.append(
                5*math.exp((-x[0]**2)) + 7*math.exp((-4*y[0]**2))
            )
        full_result.append(row_result)
    return torch.tensor(full_result)

In [ ]:
array_len = 500
x_ax = np.linspace(-5., 5., array_len)
y_ax = np.linspace(-5., 5., array_len)
z=bellcurve_2d(x_ax, y_ax, x_ax_len=array_len, y_ax_len=array_len)

data = go.Contour(
    z=z,
    x=x_ax,
    y=y_ax,
    line_smoothing=0.85,
    contours=dict(
        coloring ='heatmap',
        showlabels = True,
        labelfont = dict(size=12, color='white')
    )
)

fig = go.Figure(data=data)
fig.update_layout(title="Example 2D")
fig.show()

In [ ]:
def gen_initial_data(n=10):
    train_x = torch.rand(n, 1) # [n x 1 x 1]
    exact_obj = target_function1d(train_x).unsqueeze(-1) # [n x 1 x 1]
    best_observed_value = exact_obj.max().item()
    return train_x, exact_obj, best_observed_value

In [ ]:
def gen_next_points(init_x, init_y, best_init_y, bounds, n_points=1):
    single_model = SingleTaskGP(init_x, init_y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)

    EI = qExpectedImprovement(model=single_model, best_f=best_init_y)

    candidates, _ = optimize_acqf(acq_function=EI,
                                  bounds=bounds,
                                  q=n_points,
                                  num_restarts=200,
                                  raw_samples=512,
                                  options={"batch_limit": 5, "maxiter": 200})
    return candidates

In [ ]:
n_runs = 16

init_x, init_y, best_init_y = gen_initial_data(20)
bounds = torch.tensor([[0.], [10.]])

for i in range(n_runs):
    print(f"Nr. of optimization run: {i}")
    new_candidates = gen_next_points(init_x, init_y, best_init_y, bounds, 1)
    new_results = target_function1d(new_candidates).unsqueeze(-1)

    print(f"New candidate is: {new_candidates}")
    init_x = torch.cat([init_x, new_candidates])
    init_y = torch.cat([init_y, new_results])
    best_init_y = init_y.max().item()
    print(f"Best point performs this way: {best_init_y}")